In [2]:
# TensorFlow and tf.keras
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from google.colab import files
import google.colab as colab

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls "/content/drive/My Drive/Colab Notebooks"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
'Convolution example.ipynb'   Untitled1.ipynb   Untitled4.ipynb
 data			      Untitled2.ipynb
 Untitled0.ipynb	      Untitled3.ipynb


In [0]:
def load_dataset(path):
    import os
    import glob
    import gzip
    import numpy as np

    """Load data from `path`"""
    #path = os.path.join("/content/drive/My Drive/Colab Notebooks/", path)
    print(os.listdir(path))
    
    filelist = [x for x in os.listdir(path) if '_' in x and '.txt' in x]

    labels = [filename.split('_')[1:3] for filename in filelist]
    
    images = np.array([np.array(np.loadtxt(os.path.join(path, '/', fname))) for fname in filelist])

    return images, labels

In [11]:
images, labels, = load_dataset("/content/drive/My Drive/Colab Notebooks/data/galaxies")
images = np.asarray(images)
labels = np.asarray(labels)

#Normalize data
max_l, max_w, max_theta = 200, 100, 360
images = images / 255.
print(np.shape(labels))
print(np.shape(images))
labels[:,0], labels[:,1], labels[:,2] = labels[:,0] / max_l, labels[:,1] / max_w, labels[:,2] / max_theta

#Split into test and training sets
percent_validate = 0.2
test_images, test_labels = images[int(len(images) * percent_validate) : int(len(images))], labels[int(len(labels) * percent_validate) : int(len(labels))]
train_images, train_labels = images[0 : int(len(images) * percent_validate)], labels[0: int(len(labels) * percent_validate)]

print(np.shape(test_images), np.shape(test_labels))
print(np.shape(train_images), np.shape(train_labels))

['im5_22_80_339.txt', 'im0_41_75_179.txt', 'im1_82_16_54.txt', 'im2_90_66_147.txt', 'im3_144_95_237.txt', 'im4_187_48_121.txt']


OSError: ignored

In [0]:
#Show a sample of data
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])

In [0]:
#Loss function for multivariate regression NN
def weighted_binary_crossentropy(target, output):
    import tensorflow as tf
    import keras.backend.tensorflow_backend as tfb
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    POS_WEIGHT = 10
    
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)


In [6]:
model = keras.Sequential()
model.add(keras.layers.Dense(28*28, input_shape=(28*28,) ))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, input_shape=(256,)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, input_shape=(64,)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))
model.compile(loss=weighted_binary_crossentropy, optimizer='adam')

model.summary()

NameError: ignored

In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [0]:
#Train model
model.fit(train_images, train_labels, epochs=4, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

In [0]:
predictions = model.predict(test_images)
numpy.savetxt("results.csv", predictions, delimiter=",")